In [1]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 62.6 MB/s eta 0:00:00:00:01


In [9]:
import pdfplumber



{'matrix': (1, 0, 0, 1, 24.0, 806.749878), 'fontname': 'SRQUBF+SFUIDisplay-Bold', 'adv': 7.0954101621, 'upright': True, 'x0': 24.0, 'y0': 803.9277100370999, 'x1': 31.095410162100002, 'y1': 815.6277100371, 'width': 7.095410162100002, 'height': 11.700000000000045, 'size': 11.700000000000045, 'mcid': None, 'tag': None, 'object_type': 'char', 'page_number': 1, 'ncs': 'DeviceRGB', 'text': 'T', 'stroking_color': None, 'stroking_pattern': None, 'non_stroking_color': (0.15294117647058825, 0.43137254901960786, 0.9450980392156862), 'non_stroking_pattern': None, 'top': 26.262304962899975, 'bottom': 37.96230496290002, 'doctop': 26.262304962899975}


In [45]:
import pickle

with open('/kaggle/input/pickles/paths_order.pkl','rb') as f:
    file_paths = pickle.load(f)
with open('/kaggle/input/pickles/json_qwen_vlm.pkl','rb') as f:
    json_qwen_vlm = pickle.load(f)
with open('/kaggle/input/pickles/texts_ocr.pkl','rb') as f:
    texts_ocr = pickle.load(f)


In [48]:
texts_ocr[0]

'TAX INVOICE ORIGINAL FOR RECIPIENT\nUNCUE DERMACARE PRIVATE LIMITED\n\nGSTIN 23AADCU2395N1ZY\n\nC/o KARUNA GUPTA KURELE, 1st Floor\n\nS.P Bungalow Ke Pichhe, Shoagpur Shahdol, Shahdol\nShahdol, MADHYA PRADESH, 484001\n\nMobile +91 8585960963 Email ruhi@dermagq.in\n\nInvoice #: INV-135 Invoice Date: 01 Mar 2024 Due Date: 01 Mar 2024\n\nCustomer Details:\nMohith Saragur\n\nPlace of Supply:\n23-MADHYA PRADESH\n\n# Item Rate / Item Qty Taxable Value Tax Amount Amount\n\n1 Tab flucon 400m 22.58 3 STRP 67.75 12.19 (18%) 79.94\n9 28.23 (-20%) . , ° ,\n\n2 Lupizol ZS Shampoo 100 ml 369.91 1 BTL 369.91 44.39 (12%) 414.30\nP P 420.36 (-12%) : . ° .\n\n3 Anaboom AD Lotion - 50 ml 253.56 1 BTL 253.56 45.64 (18%) 299.20\n288.14 (-12%) : ° ° .\n\nTaxable Amount %691.22\n\nCGST 6.0% %22.19\n\nSGST 6.0% %22.19\n\nCGST 9.0% %28.92\n\nSGST 9.0% %28.92\n\nRound Off -0.45\n\nTotal ~793.00\n\nTotal Discount 117.28\n\nTotal Items / Qty : 3 / 5.000 Total amount (in words): INR Seven Hundred And Ninety-Three

In [42]:
files_path =[]

for i in file_paths:
    files_path.append(i.split('/')[-1].split('_page')[0]+'.pdf')

In [43]:
import os
pdf_directory = '/kaggle/input/pdffiles/Jan to Mar/'
all_texts = []
for filename in files_path:
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, filename)

    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[0]
        all_texts.append(first_page.extract_text())


In [156]:
def format_match(x):
    if type(x)==float or type(x)==int:
        if x>=1000:
            return str(int(x//1000))+','+str(x%1000)
        else:
            return str(x)
    else :
        return str(x)

In [158]:
acc_field_qwen = []
acc_field_ocr = []
for i in range(len(json_qwen_vlm)):
    acc_dic_qwen= {}
    acc_dic_ocr = {}
    keys = json_qwen_vlm[i].keys()
    for j in keys:
        if type(json_qwen_vlm[i][j]) == str:
            if json_qwen_vlm[i][j] in all_texts[i]:
                acc_dic_qwen[j] = 1
            else :
                acc_dic_qwen[j] = 0 
            #if json_qwen_vlm[i][j] in texts_ocr[i]:
            #    acc_dic_ocr[json_qwen_vlm[i][j]] = 1
            #else :
            #    acc_dic_ocr[json_qwen_vlm[i][j]] = 0
        if type(json_qwen_vlm[i][j]) == dict:
            dd = json_qwen_vlm[i][j]
            for k in list(dd.keys()):
                if format_match(dd[k]) in all_texts[i]:
                    acc_dic_qwen[j+k] = 1
                else :
                    acc_dic_qwen[j+k] = 0
        
        if type(json_qwen_vlm[i][j]) == list:
            dd = json_qwen_vlm[i][j]
            for k in dd:
                for l in k:
                    if format_match(k[l]) in all_texts[i]:
                        acc_dic_qwen[j+l] = 1
                    else :
                        acc_dic_qwen[j+l] = 0
    acc_field_qwen.append(acc_dic_qwen)


In [149]:
psa = [list(acc_field_qwen[i].values()).count(0) for i in range(len(acc_field_qwen))]

In [175]:
[list(acc_field_qwen[i].values()).count(0) for i in range(len(acc_field_qwen))]

[1, 2, 1, 5, 4, 1, 1, 2, 2, 3, 2, 1, 2, 2, 1, 3, 4, 2, 2, 1, 3, 2, 2, 3]

In [160]:
accuracy_percentage_each_pdf = []

for i in range(len(acc_field_qwen)):
    corr = list(acc_field_qwen[i].values()).count(1)+1 ## since address is multiline we are discounting that
    wrong = list(acc_field_qwen[i].values()).count(0)-1 ## same address count is incorrect for each of the values
    perc = corr*100/(corr+wrong)
    accuracy_percentage_each_pdf+=[perc]

In [161]:
accuracy_percentage_each_pdf

[100.0,
 96.42857142857143,
 100.0,
 85.71428571428571,
 89.28571428571429,
 100.0,
 100.0,
 96.55172413793103,
 96.55172413793103,
 93.33333333333333,
 96.42857142857143,
 100.0,
 96.42857142857143,
 96.15384615384616,
 100.0,
 93.33333333333333,
 89.28571428571429,
 96.42857142857143,
 96.55172413793103,
 100.0,
 93.10344827586206,
 96.42857142857143,
 96.42857142857143,
 92.85714285714286]

In [163]:
sum(accuracy_percentage_each_pdf)/len(accuracy_percentage_each_pdf)

95.88722580101889

In [165]:
## It does have some false negatives as we are not taking any chance with evaluation check for 99 percent so real accuracy stands at 97 + percent
##
## e.g., checking the lowest accuracy one 
low = accuracy_percentage_each_pdf.index(min(accuracy_percentage_each_pdf))
print(low)

3


In [167]:
# 'item_detailsitem_name': 0,
# this is false negative as all item names match maybe a difference of space of '-'

# 'item amount also matches for both items' # another false positive
# 

'1,754.50'

In [174]:
# total amount
json_qwen_vlm[low]['total_details']['total_amount_to_be_paid'] 
# true negative 
# real amount as in text is 1754.0

'1,754.50'

In [ ]:
# IFSC code is incorrect so it is True negative 
# In total even our lowest accuracy is 

In [186]:
# predicted accuracy for lowest is 
print(1- (list(acc_field_qwen[low].values()).count(0)-1)/len(acc_field_qwen[low]))
# real accuracy is 
print(1- (list(acc_field_qwen[low].values()).count(0)-3)/len(acc_field_qwen[low]))

0.8571428571428572
0.9285714285714286


In [176]:
json_qwen_vlm[low]

{'supplier_details': {'name': 'UNCUE DERMACARE PRIVATE LIMITED',
  'gstin_no': '23AADCU2395N1ZY',
  'address': 'C/o KARUNA GUPTA KURELE, 1st Floor S.P Bungalow Ke Pichhe, Shoagpur Shahdol, Shahdol Shahdol, MADHYA PRADESH, 484001',
  'mobile_no': '+91 8585960963',
  'email': 'ruhi@dermaq.in'},
 'invoice_details': {'invoice_no': 'INV-141',
  'invoice_date': '06 Mar 2024',
  'due_date': '06 Mar 2024'},
 'customer_details': {'name': 'Kasturi Kalwar',
  'place_of_supply': '23-MADHYA PRADESH'},
 'item_details': [{'item_no': 1,
   'item_name': 'Acne-UV Gel - spf 50 (50 gm)',
   'unit_price': '775.42',
   'unit_price_discount': '-20%',
   'quantity': '1 PAC',
   'taxable_value': '620.34',
   'tax_amount': '111.66',
   'amount': '732.00'},
  {'item_no': 2,
   'item_name': 'Bioderma Sebium Hydra Moisturiser - 40 ml',
   'unit_price': '961.86',
   'unit_price_discount': '-10%',
   'quantity': '1',
   'taxable_value': '865.68',
   'tax_amount': '155.82',
   'amount': '1021.50'}],
 'total_details':

In [169]:
acc_field_qwen[low]

{'supplier_detailsname': 1,
 'supplier_detailsgstin_no': 1,
 'supplier_detailsaddress': 0,
 'supplier_detailsmobile_no': 1,
 'supplier_detailsemail': 1,
 'invoice_detailsinvoice_no': 1,
 'invoice_detailsinvoice_date': 1,
 'invoice_detailsdue_date': 1,
 'customer_detailsname': 1,
 'customer_detailsplace_of_supply': 1,
 'item_detailsitem_no': 1,
 'item_detailsitem_name': 0,
 'item_detailsunit_price': 1,
 'item_detailsunit_price_discount': 1,
 'item_detailsquantity': 1,
 'item_detailstaxable_value': 1,
 'item_detailstax_amount': 1,
 'item_detailsamount': 0,
 'total_detailstotal_taxable_amount': 1,
 'total_detailsroundoff': 1,
 'total_detailstotal_amount_to_be_paid': 0,
 'total_detailstotal_discount': 1,
 'cgst_sgstpercentage': 1,
 'cgst_sgstamount': 1,
 'bank_detailsbank_name': 1,
 'bank_detailsaccount_no': 1,
 'bank_detailsifsc_code': 0,
 'bank_detailsbranch_name': 1}

In [170]:
print(all_texts[low])

TAX INVOICE ORIGINAL FOR RECIPIENT
UNCUE DERMACARE PRIVATE LIMITED
GSTIN 23AADCU2395N1ZY
C/o KARUNA GUPTA KURELE, 1st Floor
S.P Bungalow Ke Pichhe, Shoagpur Shahdol, Shahdol
Shahdol, MADHYA PRADESH, 484001
Mobile +91 8585960963 Email ruhi@dermaq.in
Invoice #: INV-141 Invoice Date: 06 Mar 2024 Due Date: 06 Mar 2024
Customer Details:
Kasturi Kalwar
Place of Supply:
23-MADHYA PRADESH
# Item Rate / Item Qty Taxable Value Tax Amount Amount
620.34
1 Acne-UV Gel - spf 50 (50 gm) 1 PAC 620.34 111.66 (18%) 732.00
775.42 (-20%)
Bioderma Sebium Hydra Moisturiser - 865.68
2 1 865.68 155.82 (18%) 1,021.50
40 ml 961.86 (-10%)
Taxable Amount ₹1,486.02
CGST 9.0% ₹133.74
SGST 9.0% ₹133.74
Round Off 0.50
Total ₹1,754.00
Total Discount ₹296.50
Total Items / Qty : 2 / 2.000 Total amount (in words): INR One Thousand, Seven Hundred And Fifty-Four Rupees Only.
Amount Paid
Pay using UPI: Bank Details:
For UNCUE DERMACARE PRIVATE LIMITED
Bank: Kotak Mahindra Bank
Account #: 1146860541
IFSC Code: kkbk0000725
Br

{'supplier_detailsname': 1,
 'supplier_detailsgstin_no': 1,
 'supplier_detailsaddress': 0,
 'supplier_detailsmobile_no': 1,
 'supplier_detailsemail': 1,
 'invoice_detailsinvoice_no': 1,
 'invoice_detailsinvoice_date': 1,
 'invoice_detailsdue_date': 1,
 'customer_detailsname': 1,
 'customer_detailsplace_of_supply': 1,
 'item_detailsitem_no': 1,
 'item_detailsitem_name': 1,
 'item_detailsunit_price': 0,
 'item_detailsunit_price_discount': 1,
 'item_detailsquantity': 1,
 'item_detailstaxable_value': 0,
 'item_detailstax_amount': 1,
 'item_detailsamount': 0,
 'total_detailstaxable_amount': 0,
 'total_detailsroundoff': 1,
 'total_detailstotal_amount_to_be_paid': 0,
 'total_detailstotal_discount': 1,
 'cgst_sgstpercentage': 1,
 'cgst_sgstamount': 1,
 'bank_detailsbank_name': 1,
 'bank_detailsaccount_no': 1,
 'bank_detailsifsc_code': 1,
 'bank_detailsbranch_name': 1}